 Import library

In [ ]:
from tqdm import tqdm
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras import layers
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import applications

Download kaggle Dataset

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
os.environ['KAGGLE_USERNAME']="esanjaya3"
os.environ['KAGGLE_KEY']="cc7b37b82c02b069503dc8957e22831e"

! kaggle competitions download -c pre-product-matching-id-ndsc-2020 --force
! kaggle competitions download -c product-matching-id-ndsc-2020

     |████████████████████████████████| 61kB 6.5MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.9-cp36-none-any.whl size=73265 sha256=30874e3764394c6f4d42d38105a190b65a1f364750d3b404697f268cb8ef7636
  Stored in directory: /root/.cache/pip/wheels/68/6d/9b/7a98271454edcba3b56328cbc78c037286e787d004c8afee71
Successfully built kaggle
  Found existing installation: kaggle 1.5.9
    Uninstalling kaggle-1.5.9:
      Successfully uninstalled kaggle-1.5.9
 99% 0.99G/0.99G [00:06<00:00, 202MB/s]
100% 0.99G/0.99G [00:06<00:00, 157MB/s]
100% 2.64G/2.64G [00:59<00:00, 45.2MB/s]
100% 2.64G/2.64G [00:59<00:00, 47.8MB/s]


Extract Data


In [ ]:
local_zip = '/content/pre-product-matching-id-ndsc-2020.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/product')
zip_ref.close()

local_zip = '/content/product-matching-id-ndsc-2020.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/productTest')
zip_ref.close()

Split Image in Training Set to Compare (image_1 & image_2)

In [ ]:
train_csv = pd.read_csv('/content/product/new_training_set.csv')
img_name1 = np.array(train_csv['image_1'])
img_name2 = np.array(train_csv['image_2'])
label = np.array(train_csv['Label'])

Getting Test Data From CSV file

In [ ]:
test_csv = pd.read_csv('/content/productTest/new_test_set.csv')
!ls /content/productTest

new_test_set.csv  new_training_set.csv	test_img  training_img


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Define Function for Image Preprocessing

In [ ]:
def image_load(path,list):
  train_img = []
  for im in tqdm(list):
    fnpath = os.path.join(path,im)
    img = cv2.imread(fnpath)
    img = cv2.resize(img,(75,75))
    img = img.astype('float32')
    train_img.append(img)
  train_img = np.array(train_img) 
  return train_img

Define Path Image Folder for Training and Test Dataset

In [ ]:
train_path = "/content/product/training_img/training_img/"
test_path = "/content/productTest/test_img/test_img/"

Image Preprocessing

In [ ]:
#left training image
train_img1 = image_load(train_path,train_csv["image_1"])

100%|██████████| 10181/10181 [01:27<00:00, 116.30it/s]


In [ ]:
#right training image
train_img2 = image_load(train_path,train_csv["image_2"])

100%|██████████| 10181/10181 [01:16<00:00, 133.63it/s]


In [ ]:
#left test image
test_img1 = image_load(test_path,test_csv["image_1"])

100%|██████████| 32580/32580 [04:33<00:00, 119.26it/s]


In [ ]:
#right test image
test_img2 = image_load(test_path,test_csv["image_2"])

100%|██████████| 32580/32580 [04:37<00:00, 117.61it/s]


In [ ]:
print(train_img1.shape)
print(train_img2.shape)
print(test_img1.shape)
print(test_img2.shape)

(10181, 75, 75, 3)
(10181, 75, 75, 3)
(32580, 75, 75, 3)
(32580, 75, 75, 3)


In [ ]:
# base.summary()

Model Network I

In [ ]:
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
model = Sequential()
kernel_size = 3
nb_filter = [6, 12]



#convolutional layer 1
model.add(Convolution2D(nb_filter[0], kernel_size, kernel_size, input_shape=(75,75,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(.25))

#convolutional layer 2
model.add(Convolution2D(nb_filter[1], kernel_size, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(.25))

#flatten 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(50, activation='relu'))

model.summary()

pre_img1 = model(train_img1[0:10000])
pre_img2 = model(train_img2[0:10000]) 
pretest_img1 = model(test_img1)
pretest_img2 = model(test_img2) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 6)         168       
_________________________________________________________________
activation (Activation)      (None, 25, 25, 6)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 12)          660       
_________________________________________________________________
activation_1 (Activation)    (None, 4, 4, 12)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 12)          0

Model Network II


In [ ]:
input = keras.backend.abs(pre_img1 - pre_img2)
test = keras.backend.abs(pretest_img1 - pretest_img2)
input_shape = input.shape[1]
model2 = keras.Sequential(
    [
     layers.Input(shape=(input_shape)),
     layers.Dense(1, activation="sigmoid"),
    ]
)
model2.compile(loss=keras.losses.BinaryCrossentropy(),
      optimizer=keras.optimizers.Adam(),
      metrics=['accuracy'])
fit = model2.fit(input, label[0:10000],epochs=300)

Epoch 1/300
313/313 [==============================] - 1s 2ms/step - loss: 2.8806 - accuracy: 0.5341
Epoch 2/300
313/313 [==============================] - 1s 2ms/step - loss: 1.4905 - accuracy: 0.5891
Epoch 3/300
313/313 [==============================] - 1s 2ms/step - loss: 0.8403 - accuracy: 0.6403
Epoch 4/300
313/313 [==============================] - 1s 2ms/step - loss: 0.6526 - accuracy: 0.6767
Epoch 5/300
313/313 [==============================] - 1s 2ms/step - loss: 0.6098 - accuracy: 0.6918
Epoch 6/300
313/313 [==============================] - 1s 2ms/step - loss: 0.5956 - accuracy: 0.6929
Epoch 7/300
313/313 [==============================] - 1s 2ms/step - loss: 0.5871 - accuracy: 0.7059
Epoch 8/300
313/313 [==============================] - 1s 2ms/step - loss: 0.5836 - accuracy: 0.6967
Epoch 9/300
313/313 [==============================] - 1s 2ms/step - loss: 0.5773 - accuracy: 0.6974
Epoch 10/300
313/313 [==============================] - 1s 2ms/step - loss: 0.5736 - accura

In [ ]:
pred_score = model2.predict(input)
score = np.zeros(pred_score.shape[0])
true = 0

for i in range(0,pred_score.shape[0]):

  if pred_score[i] > 0.8 :
    score[i] = 1

  if score[i] == label[i]:
    true = true + 1

acc = true/10000
print(acc)

0.6893


In [ ]:
pred_score = model2.predict(test)
score = np.zeros(pred_score.shape[0])
true = 0

for i in range(0,pred_score.shape[0]):
  if pred_score[i] > 0.6 :
    score[i] = 1



In [ ]:
score

array([1., 0., 0., ..., 0., 1., 0.])

In [ ]:
df = pd.DataFrame(score)
df.to_csv('TestProductNDSC.csv',index_label="pair_index",header=["label"])